# Question 1
The dataset can be used to:
1. Create a recommendation system that suggests products to a customer based on his previous orders.
2. Create a recommendation system that suggests the price of a new product being registered in the system.
3. Customer churn prediction.
4. Develop a system that forecasts the revenue of a company based on the price of its products and the quantity of sales in last months.
5. Develope a recommendation system that suggests new sales channels to a new businessman of a segment.

# Question 2
I have chosen the use case #3. By building a statistical model which is interpretable enough to not only predict customer churn, but also to expose the reasons that are likely to influence the customer to churn or stay, would provide information to the business to strengthen the actions that are retaining the customer and create new ones in order to overcome the weaknesses that are causing them to churn. Consequently, it will generate a reduction in the revenue loss.

# Question 3
The source code is avaible [here](https://github.com/TiagoMHCSantana/totvs-labs-challenge/blob/master/challenge.py)

# Question 4.1

## Imports

In [1]:
import numpy as np #v1.16.2
import pandas as pd #v0.24.2
from sklearn.impute import SimpleImputer #v0.20.3
from sklearn.model_selection import train_test_split, GridSearchCV #v0.20.3
from sklearn.metrics import cohen_kappa_score, classification_report #v0.20.3
import xgboost as xgb #v0.80
import scipy.special as scp #v1.2.1
import urllib2 as url #Python standard library - python v2.7.16
from zipfile import ZipFile #Python standard library - python v2.7.16
from StringIO import StringIO #Python standard library - python v2.7.16
import datetime #Python standard library - python v2.7.16
import time #Python standard library - python v2.7.16

## Definition of Functions to Extract Features
The first function is used to transform the register date of the orders into the number of days since that date. It is easier to handle a numeric value than a datetime object, while the ordinal nature of the dates is kept.

The 'compute_features(df)' function calculates and extracts some features for each customer, including the categorical features, like 'segment_code' and 'group_code', and the numerical ones, which are all computed from the attibutes 'quantity', 'total_price' and 'unit_price' of the dataset and from the attribute 'days_since_register_date' computed by the first function.

Also the dependent variable (the attribute 'is_churn') is mapped for each customer and returned by the second function. Later on, it will be separated from the independent variables.

In [2]:
def extract_days_since_register_date(df, column):
    df['days_since_'+column] = df[column].apply(lambda x: (datetime.datetime.now() - x).days)

def compute_features(df):
    #Create a new Data Frame in order to keep the calculated features.
    X = pd.DataFrame()

    #These columns are the same in all register of the same customer, so mean is applied just to group them.
    #Except unit_price, which is the price of a item bought by the customer.
    X[['is_churn', 'segment_code', 'group_code', 'average_unit_price']] =\
    df.groupby('customer_code').mean().reset_index()[['is_churn', 'segment_code', 'group_code', 'unit_price']]

    #Calculate the average quantity of items by order and total price of orders for each customer.
    X[['average_quantity_by_order', 'average_total_price_by_order']] =\
    df.groupby(['order_id', 'customer_code']).sum().groupby(level=1).mean().reset_index()[['quantity', 'total_price']]

    #Calculate the most expensive item bought by each customer.
    X['max_unit_price'] = df.groupby('customer_code').max().reset_index()['unit_price']

    #Calculate the greatest number of items and the highest price paid by each customer in a single order.
    X[['max_quantity_by_order', 'max_total_price_by_order']] =\
    df.groupby(['order_id', 'customer_code']).sum().groupby(level=1).max().reset_index()[['quantity', 'total_price']]

    #Calculate the number of orders by customer.
    X['total_orders'] = df.groupby(['customer_code', 'order_id']).mean().reset_index().groupby('customer_code').size()

    #Calculate the number of days since last order.
    X['days_since_last_order'] = df.groupby('customer_code').min().reset_index()['days_since_register_date']

    #Calculate the number of days since the first order.
    X['days_since_first_order'] = df.groupby('customer_code').max().reset_index()['days_since_register_date']

    #Calculate the frequency of orders registered by each customer expressed as the number of orders divided by the
    #	time interval from the first to the last order.
    #Multiply by 100 just to avoid precision issues.
    X['order_frequency'] = X['total_orders'] / (X['days_since_first_order'] - X['days_since_last_order'] + 1) * 100

    return X

## Dataset Reading
The dataset is read directly from the repository into a memory buffer and, then, unzipped. Using Pandas, the json file is read and parsed to a DataFrame.

In [3]:
#Reading and extracting dataset.
reader = url.urlopen('https://github.com/totvslabs/datachallenge/raw/master/challenge.zip')
file = ZipFile(StringIO(reader.read()))
json_file = file.open('challenge.json')

#Creating dataframe with dataset.
df = pd.read_json(json_file)

print df.head()

   branch_id  customer_code  group_code  is_churn  item_code  \
0          0            143           0       0.0        854   
1          0            433           0       0.0        246   
2          0            486           0       0.0       1420   
3          0            107           0       0.0       1963   
4          0            768           0       0.0       1786   

   item_total_price  order_id  quantity         register_date  sales_channel  \
0            292.91     21804        10  2017-11-10T00:00:00Z              0   
1            287.19      5486        20  2011-05-16T00:00:00Z              1   
2            184.84     22662        12  2018-01-24T00:00:00Z              0   
3            189.18      3956        18  2010-07-28T00:00:00Z              1   
4             66.87      4730         5  2010-12-17T00:00:00Z              1   

   segment_code  seller_code  total_price  unit_price  
0             0          190      1613.53       25.04  
1             5       

## Preprocessing
The first step in the preprocessing is to cast the date objects into datetimes using Pandas in order to allow for accessing the 'days' attribute. Then, the dates are transformed into days as explained before.

In the following, the unused variables are dropped in order to remove useless calculations in the feature extraction step.

The column 'is_churn' is the only column with missing values and it is used as the label of the classes (or dependent variable Y), so it is a binary classification and the mean strategy would not be suitable because it would create a third class.

The most frequent strategy is adopted due to the imbalance between the classes (over 80% of the samples are from class 0, or clients that did not churn), what makes the probability of filling in the values correctly extremely high.

Another possibility would be using clustering on the other features of the vectors with non-missing values, so as to create two cluster and, then, predict in which cluster the vectors with missing values are. This information could be used as the value missing. But this would require handling the categorical features, normalizing all features and finding an appropriate clutering method. And this task of finding a suitable method could even involve developing a specific distance metric, since the categorical features do not lay in an Euclidean space.

In [4]:
#Handling datetime data format.
df['register_date'] = pd.to_datetime(df['register_date'], format='%Y-%m-%dT%H:%M:%SZ')
extract_days_since_register_date(df, 'register_date')

#Dropping unused variables in order to remove useless calculations in the feature extraction step.
df = df.drop(['branch_id', 'seller_code', 'item_total_price', 'register_date'], axis=1)

#Columns with missing values
print 'Columns with missing values: ' + str(df.columns.to_numpy()[np.any(np.isnan(df.to_numpy()), axis=0)])

#Classes imbalance
print '\nOriginal data\n--------------'
print 'Classes: ' + str(df['is_churn'].unique())
print 'Proportion of the class 0: %.2f' % (100.0 * len(df['is_churn'][df['is_churn'] == 0.0]) / len(df['is_churn']))
print 'Proportion of the class 1: %.2f' % (100.0 * len(df['is_churn'][df['is_churn'] == 1.0]) / len(df['is_churn']))
print 'Proportion of missing values: %.2f' % (100.0 * len(df['is_churn'][df['is_churn'].isna()]) / len(df['is_churn']))

#Filling in missing values.
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

df.loc[:,:] = imputer.fit_transform(df)

print '\nAfter handling missing data\n-----------------------------'
print 'Classes: ' + str(df['is_churn'].unique())
print 'Proportion of the class 0 : %.2f' % (100.0 * len(df['is_churn'][df['is_churn'] == 0.0]) / len(df['is_churn']))
print 'Proportion of the class 1 : %.2f' % (100.0 * len(df['is_churn'][df['is_churn'] == 1.0]) / len(df['is_churn']))

Columns with missing values: [u'is_churn']

Original data
--------------
Classes: [ 0.  1. nan]
Proportion of the class 0: 80.13
Proportion of the class 1: 18.94
Proportion of missing values: 0.94

After handling missing data
-----------------------------
Classes: [0. 1.]
Proportion of the class 0 : 81.06
Proportion of the class 1 : 18.94


## Feature Extraction
Here the function defined above is simply called.

In [5]:
#Feature extraction.
X = compute_features(df)

print X.head()

   is_churn  segment_code  group_code  average_unit_price  \
0       0.0           0.0         0.0           67.850670   
1       0.0           0.0         0.0           54.524862   
2       0.0           0.0         0.0           48.270912   
3       0.0           0.0         0.0           78.777307   
4       0.0           0.0         0.0           39.980399   

   average_quantity_by_order  average_total_price_by_order  max_unit_price  \
0                  87.684211                  41351.434737          641.45   
1                  76.595238                  25800.491429          372.51   
2                  82.153846                  21577.883077          240.88   
3                 160.698413                  80996.606190         4876.80   
4                 106.902439                  26264.655122          615.67   

   max_quantity_by_order  max_total_price_by_order  total_orders  \
0                  186.0                  89396.85            19   
1                  167.0    

And the names of the features are kept before casting the dataframe into a numpy array. The names will be useful later to show to the user what is the name of the feature that most contributed to the result predicted for each customer.

In [6]:
#Keep feature names to indentify the likely reasons later on. Exclude 'is_churn' from the list.
feature_names = [str(name).replace('_', ' ') for name in X.columns.tolist()][1:]

print 'Features\n---------'
for i, feature_name in enumerate(feature_names):
    print ('%d : %s') % (i, feature_name)

Features
---------
0 : segment code
1 : group code
2 : average unit price
3 : average quantity by order
4 : average total price by order
5 : max unit price
6 : max quantity by order
7 : max total price by order
8 : total orders
9 : days since last order
10 : days since first order
11 : order frequency


## Splitting the Dataset
The dataset is split in train and test sets before hyper parameters optimization. It was chosen to hold out 25% of the samples to evaluate the model, once the total number of samples is not very large. So, 25% lefts a good percentage of the samples for training the model and also allows for model evaluation. Due to the imbalance between the classes, the splitting is stratified so that the train and test sets have samples in the same proportion of each class. Otherwise, the splitting could result in one of the sets having very few samples of the positive class, harming the final model.

In [7]:
#Separate features and labels.
y = X['is_churn'].to_numpy()
X = X.drop(['is_churn'], axis=1).to_numpy()

#Split train and test sets, 25% for test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0, stratify=y)

print 'Train data shape: ' + str(X_train.shape)
print 'Train labels shape: ' + str(y_train.shape)
print 'Test data shape: ' + str(X_test.shape)
print 'Train labels shape: ' + str(y_test.shape)

Train data shape: (628L, 12L)
Train labels shape: (628L,)
Test data shape: (210L, 12L)
Train labels shape: (210L,)


## Model Selection
The model chosen to predict customer churn is the GBM (Gradient Boosted Machines), specifically the XGBoost library was used, which is a optimized implementation of GBM. It was chosen because it is robust to feature normalization and can be used with categorical features provided that they are numeric. So, it is a model that does not require much preprocessing. Additionally, it can also return SHAP values, that expose the contribution of each feature for the values predicted. Therefore, it is also interpretable so that one can not only predict customer churn, but also can tell which is the likely reason for the customer staying or churning.

An alternative methodology for model selection would be to use nested cross-validation with some different models and then choosing the best one. But it is more time consuming and using some types of models would require to handle categorical data (e.g., using one-hot encoding) and feature normalization. Furthermore, some models are not as interpretable as the XGBoost library made the GBM model to be.

In [8]:
#Instantiate model.
xgb_model = xgb.XGBClassifier()

## Hyper Parameters Optimization
In order to tune the hyper parameters, grid search with 5-fold Cross-validation was used on the train set. The cross-validation is very useful here because the dataset resulted in a very limited number of samples of customer churn so that splitting them into three sets (train, validation and test) could eventually cause the model to underfit.

It is important to highlight that when the 'cv' parameter is an integer, the folds splitting is stratified by default in the scikit-learn library. Also the refit parameter was set to 'True' in order to refit a model using the best parameters and all training data after applying the grid search.

In [9]:
#Hyper parameters optimization through grid search with 5-fold cross-validation.
parameters = {'nthread':[4],
              'objective':['binary:hinge', 'binary:logistic'],
              'learning_rate': [0.005, 0.05],
              'max_depth': [3, 5, 7],
              'min_child_weight': [9, 11],
              'silent': [1],
              'subsample': [0.6, 0.7, 0.8],
              'colsample_bytree': [0.6, 0.7, 0.8],
              'n_estimators': [1000],
              'seed': [0]}

clf = GridSearchCV(xgb_model, parameters, n_jobs=4, 
                   cv=5,
                   scoring=None,
                   verbose=0, refit=True)

clf.fit(X_train, y_train)

print 'Best score: %.3f' % clf.best_score_
print '\nBest params\n-------------'
for param, value in clf.best_params_.iteritems():
    print ('%s : %s') % (param, str(value))

Best score: 0.952

Best params
-------------
colsample_bytree : 0.7
silent : 1
learning_rate : 0.005
nthread : 4
min_child_weight : 9
n_estimators : 1000
subsample : 0.8
seed : 0
objective : binary:hinge
max_depth : 7


## Customer Churn Prediction
The best model refitted is then used to predict customer churn. Additionaly, the contributions of each feature to the predictions are computed and mapped to the feature names in order to tell which is the likely reason for each customer to stay or churn. The features with highest and lowest SHAP values are the ones that most influenced the prediction towards the class 1 (customer churn) and class 0 (customer stay), respectively.

In [10]:
#Use best model found to predict churn.
y_pred = clf.predict(X_test)

#Get and report likely reason.
xgb_booster = clf.best_estimator_.get_booster()
contrib = xgb_booster.predict(xgb.DMatrix(X_test), pred_contribs=True)

for i, pred in enumerate(y_pred):
    if pred == 1.0:
        most_influential_reason_idx = np.argmax(contrib[i,:-1]) #the feature that most influenced positively
        client_status = 'churn'
    else:
        most_influential_reason_idx = np.argmin(contrib[i,:-1]) #the feature that most influenced negatively
        client_status = 'stay'
    print ('Customer %d will probably %s.\n\tLikely reason: %28s = %7s\t(margin contribution): %.3f') %\
          (i,
           client_status,
           feature_names[most_influential_reason_idx],
           '{:.3f}'.format(X_test[i, most_influential_reason_idx]),
           contrib[i, most_influential_reason_idx])

Customer 0 will probably stay.
	Likely reason:        days since last order = 343.000	(margin contribution): -0.181
Customer 1 will probably stay.
	Likely reason:       days since first order = 1960.000	(margin contribution): -0.173
Customer 2 will probably stay.
	Likely reason:        days since last order = 351.000	(margin contribution): -0.282
Customer 3 will probably stay.
	Likely reason:       days since first order = 1962.000	(margin contribution): -0.200
Customer 4 will probably stay.
	Likely reason:       days since first order = 3241.000	(margin contribution): -0.229
Customer 5 will probably stay.
	Likely reason:       days since first order = 3903.000	(margin contribution): -0.216
Customer 6 will probably churn.
	Likely reason:              order frequency =   6.096	(margin contribution): 1.261
Customer 7 will probably stay.
	Likely reason:       days since first order = 2321.000	(margin contribution): -0.223
Customer 8 will probably stay.
	Likely reason:        days since la

## Model Evaluation
Cohen's Kappa Score and F1-score were chosen to assess the model. Cohen's Kappa Score is a measure of agreement between raters that takes into account the likelihood of they agreeing by chance. In this case, the score is measuring the agreement between the actual labels and the predicted labels. According to a possible interpretation of the score, a value above 0.8 may be regarded as 'almost perfect' [2]. Additionally, the F1-score is used because it combines the values of precision and recall, which are very suitable to the problem being tackled: the precision shows how much a prediction of the model is reliable and the recall tells how many of the customers that will stay or churn are indentified.

In [11]:
#Report results.
print ('\nCohen Kappa Score: %.3f\n') % (cohen_kappa_score(y_test, y_pred))
print classification_report(y_test, y_pred)


Cohen Kappa Score: 0.942

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99       190
         1.0       1.00      0.90      0.95        20

   micro avg       0.99      0.99      0.99       210
   macro avg       0.99      0.95      0.97       210
weighted avg       0.99      0.99      0.99       210



## References
[1] Landis, J. R. and Koch, G. G. (1977). The measurement of observer agreement for
categorical data. Biometrics, 33(1):159--174. ISSN 0006-341X.

# Question 4.2
If I had more time, I would improve the model by thinking of a few more features and maybe working on an ensemble of different classifiers. Using an ensemble would allow me for training each classifier using just some features that are suitable to the model and then to use one type of majority vote approach to output the final prediction.